In [ ]:
import pygame
import random
import math

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
BOID_COUNT = 50
BOID_RADIUS = 5
BOID_SPEED = 2
NEIGHBOR_RADIUS = 100 #radius where boids detect each other
SEPARATION_RADIUS = 50 #minimum radius apart
COUPLING_STRENGTH = 0  # Kuramoto coupling strength
FPS = 60

# Colors
BLACK = (0, 0, 0)

# helping function to map one range to another (for coloring purposes)
def map_value(value, left_min, left_max, right_min, right_max):
    left_span = left_max - left_min
    right_span = right_max - right_min
    value_scaled = (value - left_min) / left_span
    return right_min + (value_scaled * right_span)

# coloring helper
def hsv_to_rgb(h, s, v):
    return tuple(int(c * 255) for c in pygame.Color(0).hsva_to_rgb(h, s, v))

# Boid Class
class Boid:
    def __init__(self, x, y):
        self.position = pygame.Vector2(x, y)
        self.velocity = pygame.Vector2(random.uniform(-1, 1), random.uniform(-1, 1)).normalize() * BOID_SPEED

        self.phase = math.atan2(self.velocity.y, self.velocity.x)  # Initial heading as phase
        self.natural_frequency = random.uniform(-0.1, 0.1)  # Random natural frequency for Kuramoto model

    def move(self):
        self.position += self.velocity
        # Wrap around screen edges
        if self.position.x > WIDTH:
            self.position.x = 0
        elif self.position.x < 0:
            self.position.x = WIDTH
        if self.position.y > HEIGHT:
            self.position.y = 0
        elif self.position.y < 0:
            self.position.y = HEIGHT

    def draw(self, screen):
        # Map phase to hue (0 to 360 degrees)
        hue = map_value(self.phase, -math.pi, math.pi, 0, 360)
        color = pygame.Color(0)
        color.hsva = (hue, 100, 100)  # Full saturation and brightness
        pygame.draw.circle(screen, color, (int(self.position.x), int(self.position.y)), BOID_RADIUS)


# distance
def distance(vec1, vec2):
    return math.sqrt((vec1.x - vec2.x) ** 2 + (vec1.y - vec2.y) ** 2)

# Phase-based function f
def phase_function(phase_diff):
    if 0 < phase_diff < math.pi:
        return (0.5 * phase_diff - 0.5 * math.pi) ** 2
    elif -math.pi < phase_diff <= 0:
        return (0.5 * phase_diff + 0.5 * math.pi) ** 2
    else:
        return 0  # Outside defined range or y > 2.4674

# Gaussian function g
def gaussian(distance, sigma=25):  # Adjust sigma for interaction range
    return math.exp(-distance**2 / (2 * sigma**2))


# Boid Rules
def apply_kuramoto(boid, boids, coupling_strength):
    phase_coupling = 0
    nearby_boids = 0

    for other in boids:
        if other == boid:
            continue
        if distance(boid.position, other.position) < NEIGHBOR_RADIUS:
            nearby_boids += 1
            phase_coupling += math.sin(other.phase - boid.phase)

    if nearby_boids > 0:
        # update phase using natural frequency and coupling
        boid.phase += boid.natural_frequency + (coupling_strength / nearby_boids) * phase_coupling
        # Keep phase within [-π, π] for simplicity
        boid.phase = (boid.phase + math.pi) % (2 * math.pi) - math.pi

    # Update velocity to align with new phase
    boid.velocity = pygame.Vector2(math.cos(boid.phase), math.sin(boid.phase)) * BOID_SPEED


def apply_rules(boid, boids):
    separation_force = pygame.Vector2(0, 0)
    alignment_force = pygame.Vector2(0, 0)
    cohesion_force = pygame.Vector2(0, 0)
    nearby_boids = 0

    for other in boids:
        if other == boid:
            continue
        dist = distance(boid.position, other.position)
        if dist < NEIGHBOR_RADIUS:
            nearby_boids += 1
            alignment_force += other.velocity
            cohesion_force += other.position
            if dist < SEPARATION_RADIUS:
                separation_force += boid.position - other.position

    if nearby_boids > 0:
        # Normalize and average forces
        alignment_force = (alignment_force / nearby_boids).normalize() * BOID_SPEED
        cohesion_force = ((cohesion_force / nearby_boids) - boid.position).normalize() * BOID_SPEED
        if separation_force.length() > 0:  # Avoid zero-length vector normalization
            separation_force = separation_force.normalize() * BOID_SPEED

    # Combine forces with weights
    boid.velocity += separation_force * 1.5 + alignment_force * 1.0 + cohesion_force * 1.0
    boid.velocity = boid.velocity.normalize() * BOID_SPEED  # Ensure consistent speed

    # Apply Kuramoto coupling for phase synchronization
    apply_kuramoto(boid, boids, COUPLING_STRENGTH)


# Main Function
def main():
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Boids Simulation with Kuramoto Coupling and Phase Coloring")
    clock = pygame.time.Clock()

    # Initialize Boids
    boids = [Boid(random.randint(0, WIDTH), random.randint(0, HEIGHT)) for _ in range(BOID_COUNT)]

    running = True
    while running:
        screen.fill(BLACK)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Update Boids
        for boid in boids:
            apply_rules(boid, boids)
            boid.move()
            boid.draw(screen)

        pygame.display.flip()
        clock.tick(FPS)

    pygame.quit()


# Run the Simulation
if __name__ == "__main__":
    main()


KeyboardInterrupt: 

: 